## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_count = application_df['APPLICATION_TYPE'].value_counts()
application_count

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_count[application_count < 500].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_count[classification_count > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_count[classification_count < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(['IS_SUCCESSFUL'], axis=1).values
y = application_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=48)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model (Attempt #1)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
inputs_features = len(X[0])
first_layer_nodes = 42
second_layer_nodes = 24
output_layer_nodes = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=first_layer_nodes, activation='relu', input_dim=inputs_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=second_layer_nodes, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=output_layer_nodes, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 42)             │         1,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 24)             │         1,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,905 (11.35 KB)

 Trainable params: 2,905 (11.35 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 243us/step - accuracy: 0.6896 - loss: 0.5987
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 232us/step - accuracy: 0.7286 - loss: 0.5589
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 284us/step - accuracy: 0.7310 - loss: 0.5534
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 234us/step - accuracy: 0.7300 - loss: 0.5508
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 232us/step - accuracy: 0.7316 - loss: 0.5502
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 233us/step - accuracy: 0.7279 - loss: 0.5527
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 229us/step - accuracy: 0.7364 - loss: 0.5438
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 230us/step - accuracy: 0.7338 - loss: 0.5457
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 228us/step - accuracy: 0.7370 - loss: 0.5427
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 239us/step - accuracy: 0.7331 - loss: 0.5477
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 229us/step - accuracy: 0.7285 - loss: 0.5487
Epoch 12/100
804/80

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 333us/step - accuracy: 0.7265 - loss: 0.5586
Loss: 0.5586358308792114, Accuracy: 0.7265306115150452


In [16]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization1.h5")

## Compile, Train and Evaluate the Model (Attempt #2)

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
inputs_features = len(X[0])
first_layer_nodes = 12
second_layer_nodes = 22
third_layer_nodes = 10
output_layer_nodes = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=first_layer_nodes, activation='relu', input_dim=inputs_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=second_layer_nodes, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=third_layer_nodes, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=output_layer_nodes, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 12)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 22)             │           286 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,055 (4.12 KB)

 Trainable params: 1,055 (4.12 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 242us/step - accuracy: 0.6762 - loss: 0.6202
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 234us/step - accuracy: 0.7256 - loss: 0.5574
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 233us/step - accuracy: 0.7215 - loss: 0.5626
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step - accuracy: 0.7247 - loss: 0.5567
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 237us/step - accuracy: 0.7330 - loss: 0.5529
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 237us/step - accuracy: 0.7321 - loss: 0.5468
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 234us/step - accuracy: 0.7316 - loss: 0.5492
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 256us/step - accuracy: 0.7334 - loss: 0.5454
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 296us/step - accuracy: 0.7295 - loss: 0.5505
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 238us/step - accuracy: 0.7387 - loss: 0.5432
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 236us/step - accuracy: 0.7343 - loss: 0.5472
Epoch 12/100
804/80

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 326us/step - accuracy: 0.7266 - loss: 0.5530
Loss: 0.5530306100845337, Accuracy: 0.7266472578048706


In [21]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization2.h5")

### Compile, Train and Evaluate the Model (Attempt #3)

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
inputs_features = len(X[0])
first_layer_nodes = 22
second_layer_nodes = 35
output_layer_nodes = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=first_layer_nodes, activation='relu', input_dim=inputs_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=second_layer_nodes, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=output_layer_nodes, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 22)             │           968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 35)             │           805 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            36 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,809 (7.07 KB)

 Trainable params: 1,809 (7.07 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 239us/step - accuracy: 0.6999 - loss: 0.6095
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 229us/step - accuracy: 0.7288 - loss: 0.5584
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 294us/step - accuracy: 0.7328 - loss: 0.5545
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step - accuracy: 0.7325 - loss: 0.5475
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 229us/step - accuracy: 0.7348 - loss: 0.5460
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step - accuracy: 0.7306 - loss: 0.5481
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 229us/step - accuracy: 0.7271 - loss: 0.5501
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 228us/step - accuracy: 0.7284 - loss: 0.5505
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 233us/step - accuracy: 0.7300 - loss: 0.5502
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 236us/step - accuracy: 0.7359 - loss: 0.5449
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 232us/step - accuracy: 0.7346 - loss: 0.5456
Epoch 12/100
804/80

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 330us/step - accuracy: 0.7272 - loss: 0.5538
Loss: 0.5538013577461243, Accuracy: 0.7272303104400635


In [26]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization3.h5")

#### Compile, Train and Evaluate the Model (Attempt #4)

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
inputs_features = len(X[0])
first_layer_nodes = 15
second_layer_nodes = 24
third_layer_nodes = 8
output_layer_nodes = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=first_layer_nodes, activation='relu', input_dim=inputs_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=second_layer_nodes, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=third_layer_nodes, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=output_layer_nodes, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 15)             │           660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 24)             │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 8)              │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,253 (4.89 KB)

 Trainable params: 1,253 (4.89 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 253us/step - accuracy: 0.6824 - loss: 0.6160
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 242us/step - accuracy: 0.7235 - loss: 0.5621
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step - accuracy: 0.7282 - loss: 0.5567
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 239us/step - accuracy: 0.7344 - loss: 0.5504
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 238us/step - accuracy: 0.7291 - loss: 0.5532
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 241us/step - accuracy: 0.7363 - loss: 0.5450
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step - accuracy: 0.7325 - loss: 0.5477
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 241us/step - accuracy: 0.7360 - loss: 0.5462
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step - accuracy: 0.7339 - loss: 0.5468
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step - accuracy: 0.7353 - loss: 0.5477
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 239us/step - accuracy: 0.7305 - loss: 0.5514
Epoch 12/100
804/80

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 337us/step - accuracy: 0.7293 - loss: 0.5531
Loss: 0.5531057119369507, Accuracy: 0.7293294668197632


In [31]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization4.h5")